# Imports

In [ ]:
# Import libraries
import numpy as np
import pandas as pd
import math
from time import time
from sklearn import datasets
from sklearn.metrics import adjusted_rand_score
from sklearn.model_selection import train_test_split
from sklearn import neighbors
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import f1_score

#from ucimlrepo import fetch_ucirepo, list_available_datasets

from csv import reader
from math import sqrt
import random
from random import seed
from random import randrange
import requests
import io
    
# Downloading the csv file from your GitHub account

url = "https://raw.githubusercontent.com/Zuluke/Projetos-AM/main/Projeto_Chico/Dataset/mfeat-fac" # Make sure the url is the raw version of the file on GitHub
download = requests.get(url).content
fac = pd.read_csv(io.StringIO(download.decode('utf-8')),sep="\s+",header=None)

url = "https://raw.githubusercontent.com/Zuluke/Projetos-AM/main/Projeto_Chico/Dataset/mfeat-fou" # Make sure the url is the raw version of the file on GitHub
download = requests.get(url).content
fou = pd.read_csv(io.StringIO(download.decode('utf-8')),sep="\s+",header=None)

url = "https://raw.githubusercontent.com/Zuluke/Projetos-AM/main/Projeto_Chico/Dataset/mfeat-zer" # Make sure the url is the raw version of the file on GitHub
download = requests.get(url).content
zer = pd.read_csv(io.StringIO(download.decode('utf-8')),sep="\s+",header=None)

### Adjusting the datasets


In [ ]:
#label creation

fac["digit"] = [i for i in range(10) for __ in range(200)]

fou["digit"] = [i for i in range(10) for __ in range(200)]

zer["digit"] = [i for i in range(10) for __ in range(200)]

# Functions

In [1]:
                                              # A função recebe o cluster i, e os índices l,k que correspondem a dois vetores que representam duas instâncias.
def kernel_gaussian(vector1,l,vector2,k):   # função que calcula o kernel gaussiano, equação (10) do artigo.
  kernel = vector1[l] - vector2[k]
  kernel = np.sum(np.multiply(s,np.multiply(kernel,kernel)))
  kernel = math.exp(kernel*(-1/2))
  return kernel

In [2]:
def u_ki(k,i):                #calcula os graus de pertencimento u_ik
  numerator = (2-(2*kernel_matrix[k]))
  belong = sum([(numerator/(2-(2*kernel_gaussian(x,k,g,h))))**(1/(m-1)) for h in range(c)])
  belong = belong ** -1
  return belong

def update_u():               #atualiza todos os valores de u na matriz
  for k in range(n):
    aux = 0
    for z in range(c):
      if(np.array_equal(x[k],g[z])):
        u[k] = 0
        u[k][z] = 1
        aux += 1
    if(aux==0):
      for i in range(c):
          u[k][i] = u_ki(k,i)

In [3]:
def update_J():
  objective = np.sum(np.sum(np.multiply(u_m,(2-kernel_matrix*2)),axis=0))
  return objective

In [4]:
def update_g():
  multiplied_aux = np.multiply(u_m,kernel_matrix)
  numerator = np.sum(np.multiply(multiplied_aux[:,np.newaxis],np.repeat(x[:,:, np.newaxis], c, axis=2)), axis = 0)
  denominator = np.sum(multiplied_aux, axis = 0)
  g = numerator/denominator
  g = g.transpose()
  return g

In [5]:
from decimal import Decimal, getcontext
import time

getcontext().prec = 120
def update_s():
  for i in range(c):
    multiplied_aux = np.multiply(u_m[:,i],kernel_matrix[:,i])
    for j in range(p):
      numerator = Decimal('1')
      expoent = Decimal(p)
      expoent = numerator/expoent
      for h in range(p):
        top_aux = x[:,h] - g[i,h]
        numerator_aux = np.multiply(multiplied_aux,np.multiply(top_aux,top_aux))
        numerator_aux = np.sum(numerator_aux)
        numerator_aux = Decimal(numerator_aux)
        numerator = numerator * numerator_aux
      numerator = numerator**expoent
      numerator = float(numerator)
      bot_aux = x[:,j] - g[i,j]
      denominator = np.multiply(multiplied_aux,np.multiply(bot_aux,bot_aux))
      denominator = np.sum(denominator)
      s[i,j] = numerator/denominator

In [6]:
def update_kernel():
  kernel_m = np.multiply(np.repeat(s.transpose()[np.newaxis,:,:], n, axis=0),(np.subtract(x[:,:,np.newaxis],np.repeat(g.transpose()[np.newaxis,:,:], n, axis=0)))**2)
  kernel_m = np.sum(kernel_m, axis=1) * (-1/2)
  kernel_m = np.exp(kernel_m)
  return kernel_m

In [7]:
from sklearn import datasets

iris = datasets.load_iris()
classes = iris.target
dataset = iris.data

In [8]:
import random
import time
from sklearn.preprocessing import StandardScaler

dataset = dataset

min_val = np.min(dataset)                                       #normalizando valores
max_val = np.max(dataset)
scaled_data = (dataset - min_val) / (max_val - min_val)

#scaler = StandardScaler()
#scaled_data = scaler.fit_transform(dataset)


n_clusters = 3                    #trocar pra 10 clusters quando for rodar na base do projeto
n = dataset.shape[0]  #quantidade de instâncias n
p = dataset.shape[1]  #quantidade de atributos p
c = n_clusters       #quatidade de clusters c
s = np.ones((1,p))    #inicialização da matriz que contém os parâmetros width s^2  / passo 7
u = np.ones((n,c))
u_m = np.ones((n,c))
kernel_matrix = np.ones((n,c))                     #martiz de kernels
x = scaled_data #dataset           #salva o dataset em x para concordar com a nomenclatura utilizada no artigo
m = 1.6               # hiper-parâmetro do algoritmo
epoch = 10
e = 0.000001

prototypes = random.sample(range(n), n_clusters)   # seleção randomizada dos índices dos protótipos dos clusters
g = [x[i] for i in prototypes]  # colocando no vetor g de protótipos os vetores dos protótipos selecionados a partir dos índices    / passo 8
g = np.array(g)

start_time = time.time()
kernel_matrix = update_kernel()
update_u()                       #passo 10
u_m = u**m
J_NEW = update_J()               #passo 11

for a in range(epoch):
  kernel_matrix = update_kernel()
  J_OLD = J_NEW                             #passo 13
  update_s()                                 #passo 16
  kernel_matrix = update_kernel()
  g = update_g()                                 #passo 19
  kernel_matrix = update_kernel()
  update_u()                                 #passo 22
  u_m = u**m
  J_NEW = update_J()                         #passo 23
  print(f'epoch: {a}; Função Objetivo J: {J_NEW}')
  if(abs(J_NEW-J_OLD)<e):
    break
end_time = time.time()
print(f'Tempo de execução: {end_time - start_time}')

epoch: 0; Função Objetivo J: 2.9358843629950337
epoch: 1; Função Objetivo J: 1.895956131434533
epoch: 2; Função Objetivo J: 1.2530120329151384
epoch: 3; Função Objetivo J: 1.0200408862615649
epoch: 4; Função Objetivo J: 0.9876864580897302
epoch: 5; Função Objetivo J: 0.9767945038546225
epoch: 6; Função Objetivo J: 0.972135589729852
epoch: 7; Função Objetivo J: 0.9706935743298369
epoch: 8; Função Objetivo J: 0.9706086847743953
epoch: 9; Função Objetivo J: 0.9709589037172071
Tempo de execução: 0.16960430145263672


In [9]:
from sklearn.metrics.cluster import adjusted_rand_score

pred = np.argmax(u, axis=1)
adjusted_rand_score(pred, classes)

0.8340750470114582

In [10]:
unique, counts = np.unique(pred, return_counts=True)

dict(zip(unique, counts))

{0: 53, 1: 50, 2: 47}

In [11]:
PC = np.sum(u**2)/n                                   #modified partition coeficient
MPC = 1 - (c/(c-1))*(1-PC)
MPC

0.8307910368086072